In [1]:
import numpy as np
import random
from FPU import cmul, cadd
from utils import read_binary, binary_to_fp16, store_binary, generate_fp16

In [5]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# Frame builders (standalone demo)
# -----------------------------------------------------------------------------
def complex_sinusoid(N, freq_bin=5, amp=0.3, phase=0.0):
    """
    Generate a complex sinusoid x[n] = amp * exp(j*2*pi*freq_bin*n/N + phase)
    """
    n = np.arange(N, dtype=np.float32)
    arg = 2.0 * np.pi * freq_bin * n / float(N) + phase
    re = amp * np.cos(arg)
    im = amp * np.sin(arg)
    return re, im

def add_awgn(re, im, sigma=0.05):
    """
    Add complex AWGN to real/imag arrays (independent N(0,sigma^2)).
    """
    re = re + np.random.normal(0.0, sigma, size=re.shape).astype(np.float32)
    im = im + np.random.normal(0.0, sigma, size=im.shape).astype(np.float32)
    return re, im

def ofdm_like(N, num_active=16, amp=0.8):
    """
    Simple OFDM-like burst using random subcarriers and QPSK symbols.
    """
    n = np.arange(N, dtype=np.float32)
    cand = list(range(1, N // 2))
    num_active = min(num_active, len(cand))
    active = np.random.choice(cand, size=num_active, replace=False)
    qpsk = np.array([(1,1), (1,-1), (-1,1), (-1,-1)], dtype=np.float32) / np.sqrt(2.0)
    re = np.zeros(N, dtype=np.float32)
    im = np.zeros(N, dtype=np.float32)
    scale = amp / np.sqrt(num_active)
    for k in active:
        a_re, a_im = qpsk[np.random.randint(0, len(qpsk))]
        arg = 2.0 * np.pi * k * n / float(N)
        c_re = np.cos(arg)
        c_im = np.sin(arg)
        re += scale * (a_re * c_re - a_im * c_im)
        im += scale * (a_re * c_im + a_im * c_re)
    return re, im

def chirp_linear(N, f0_bin=2, f1_bin=40, amp=0.8):
    """
    Complex linear chirp sweeping from f0_bin to f1_bin across N samples.
    """
    n = np.arange(N, dtype=np.float32)
    t = n / float(N)
    phi = 2.0 * np.pi * (float(f0_bin) * t + 0.5 * (float(f1_bin) - float(f0_bin)) * t * t)
    re = amp * np.cos(phi).astype(np.float32)
    im = amp * np.sin(phi).astype(np.float32)
    return re, im

def build_frames(num_frames=10, N=128, base_freq_bin=5, noise_sigma=0.05,
                 event_frames=1, event_type="auto", seed=123):
    """
    Build frames with base complex sinusoid + AWGN and insert OFDM/Chirp bursts.
    Returns:
        frames_re, frames_im, gt_event_idx
    """
    rng = np.random.default_rng(seed)
    frames_re = []
    frames_im = []
    gt_event_idx = sorted(rng.choice(num_frames, size=event_frames, replace=False).tolist())

    for f in range(num_frames):
        re, im = complex_sinusoid(N, freq_bin=base_freq_bin, amp=0.3, phase=rng.random() * 2 * np.pi)
        re, im = add_awgn(re, im, sigma=noise_sigma)
        if f in gt_event_idx:
            et = "OFDM" if (event_type == "auto" and rng.random() < 0.5) else ("Chirp" if event_type == "auto" else event_type)
            if et == "OFDM":
                e_re, e_im = ofdm_like(N, num_active=16, amp=0.8)
            else:
                e_re, e_im = chirp_linear(N, f0_bin=2, f1_bin=max(3, N // 3), amp=0.8)
            re = re + e_re
            im = im + e_im
        frames_re.append(re)
        frames_im.append(im)
    return frames_re, frames_im, gt_event_idx

# -----------------------------------------------------------------------------
# STFT / Spectrogram
# -----------------------------------------------------------------------------
def stft_mag_db(x, fs, n_fft=256, hop=64, window="hann"):
    """
    Compute magnitude spectrogram (in dB) using STFT.
    Returns (S_db, freqs, times)
    """
    win = np.hanning(n_fft).astype(np.float32) if window == "hann" else np.ones(n_fft, dtype=np.float32)

    if len(x) < n_fft:
        x = np.pad(x, (0, n_fft - len(x)), mode="constant")

    n_frames = 1 + int(np.ceil((len(x) - n_fft) / float(hop)))
    total_len = (n_frames - 1) * hop + n_fft
    if total_len > len(x):
        x = np.pad(x, (0, total_len - len(x)), mode="constant")

    S = np.empty((n_fft // 2 + 1, n_frames), dtype=np.float32)
    times = np.empty(n_frames, dtype=np.float32)
    for i in range(n_frames):
        start = i * hop
        frame = x[start:start + n_fft] * win
        X = np.fft.rfft(frame, n=n_fft)
        P = (np.abs(X) ** 2).astype(np.float32)
        S[:, i] = P
        times[i] = (start + n_fft / 2.0) / fs

    freqs = np.fft.rfftfreq(n_fft, d=1.0 / fs).astype(np.float32)
    S_db = 10.0 * np.log10(S + 1e-12)
    return S_db, freqs, times

# -----------------------------------------------------------------------------
# Demo run (10 frames × 128 samples)
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    NUM_FRAMES = 10
    N_PER_FRAME = 128
    FS = 100_000.0           # Sample rate for axis labeling (Hz)
    frames_re, frames_im, gt = build_frames(num_frames=NUM_FRAMES,
                                            N=N_PER_FRAME,
                                            base_freq_bin=5,
                                            noise_sigma=0.05,
                                            event_frames=1,
                                            event_type="auto",
                                            seed=123)

    # Concatenate into one complex sequence
    x_re = np.concatenate(frames_re).astype(np.float32)
    x_im = np.concatenate(frames_im).astype(np.float32)
    x = x_re + 1j * x_im



In [10]:
frames_re[0]

array([-0.07644214,  0.6606054 ,  0.30773994, -0.00797276,  0.62623304,
        0.17393064, -0.6934095 ,  0.36884317, -0.43134585,  0.10730262,
        0.5985034 , -0.95783085,  0.38601413,  0.50352496,  0.61023486,
        0.11270942, -0.46707204, -0.25125837, -0.528931  , -0.52690244,
       -0.8584003 , -1.8959299 ,  0.1893825 , -0.64310634,  0.29925013,
        0.67314446,  0.16224396,  1.3385276 ,  1.2427282 ,  0.16201457,
        0.87055457, -0.5290784 , -0.39675203, -0.82519186, -1.0005735 ,
       -0.97202146, -0.45954788, -0.90030587, -0.20145634,  0.34172273,
        0.17558582,  0.52414405, -0.11280226, -0.24736108,  0.36584324,
       -0.04361068, -0.78251636, -0.70251924, -0.2926363 ,  0.06505933,
        0.08169629,  0.25656402, -0.84002066,  0.97082746,  0.11885944,
       -0.3199211 , -0.27507108, -0.77140695, -0.77444303, -0.05589939,
       -0.97589225, -0.51193833, -0.53626287,  0.05595163, -0.59673095,
        0.33172047, -0.02788025, -0.2248792 ,  0.17922068, -0.08

In [3]:
import numpy as np
import random
from FPU import cmul, cadd
from utils import store_binary, generate_fp16, binary_to_integer, fp16_to_binary

# =============================================================================
# Signal utilities
# =============================================================================

def to_fp16_pair_array(re: np.ndarray, im: np.ndarray):
    """
    Convert real/imag float arrays to list of [np.float16(re), np.float16(im)].
    """
    re16 = re.astype(np.float16)
    im16 = im.astype(np.float16)
    out = []
    for r, i in zip(re16, im16):
        out.append([r, i])
    return out

def complex_sinusoid(N, freq_bin=5, amp=0.3, phase=0.0):
    """
    Generate a complex sinusoid: x[n] = amp * exp(j*2*pi*freq_bin*n/N + phase)
    Returned as (re, im) in float32 arrays for further mixing.
    """
    n = np.arange(N, dtype=np.float32)
    arg = 2.0 * np.pi * freq_bin * n / float(N) + phase
    re = amp * np.cos(arg)
    im = amp * np.sin(arg)
    return re, im

def add_awgn(re, im, sigma=0.05):
    """
    Add complex Gaussian noise with std=sigma to re/im separately.
    """
    re = re + np.random.normal(0.0, sigma, size=re.shape).astype(np.float32)
    im = im + np.random.normal(0.0, sigma, size=im.shape).astype(np.float32)
    return re, im

def ofdm_like(N, num_active=16, amp=0.8):
    """
    Generate a simple OFDM-like burst by summing a subset of subcarriers with QPSK symbols.
    No CP for simplicity; time-domain x[n] = sum_k a_k * exp(j*2*pi*k*n/N).
    """
    n = np.arange(N, dtype=np.float32)
    # Random unique subcarriers (avoid DC for variety)
    cand = list(range(1, N//2))
    active = random.sample(cand, k=min(num_active, len(cand)))
    # QPSK symbols: {1+j, 1-j, -1+j, -1-j} normalized
    qpsk = [(1,1), (1,-1), (-1,1), (-1,-1)]
    re = np.zeros(N, dtype=np.float32)
    im = np.zeros(N, dtype=np.float32)
    scale = amp / np.sqrt(num_active)  # keep peak in range
    for k in active:
        a_re, a_im = random.choice(qpsk)
        # Normalize symbol power to 1
        a_re = a_re / np.sqrt(2.0)
        a_im = a_im / np.sqrt(2.0)
        arg = 2.0 * np.pi * k * n / float(N)
        c_re = np.cos(arg)
        c_im = np.sin(arg)
        # (a_re + j a_im) * (c_re + j c_im)
        re += scale * (a_re * c_re - a_im * c_im)
        im += scale * (a_re * c_im + a_im * c_re)
    return re, im

def chirp_linear(N, f0_bin=2, f1_bin=40, amp=0.8):
    """
    Generate a complex linear chirp sweeping from f0_bin to f1_bin across N samples.
    Frequency bins are normalized to 1/N of sampling rate.
    """
    n = np.arange(N, dtype=np.float32)
    # Instantaneous phase for linear chirp: phi[n] = 2*pi*(f0*n/N + 0.5*(f1-f0)*(n/N)^2)
    f0 = float(f0_bin)
    f1 = float(f1_bin)
    t = n / float(N)
    phi = 2.0 * np.pi * (f0 * t + 0.5 * (f1 - f0) * t * t)
    re = amp * np.cos(phi).astype(np.float32)
    im = amp * np.sin(phi).astype(np.float32)
    return re, im

# =============================================================================
# Energy Detection (ED)
# =============================================================================

def cconj(z_pair):
    """
    Conjugate of [re, im] in np.float16 pairs.
    """
    return [z_pair[0], np.float16(-z_pair[1])]

def ed_sum_abs2(frame_pairs):
    """
    Sum of |x[n]|^2 using the provided FPU cmul/cadd primitives.
    - frame_pairs: list of [np.float16(re), np.float16(im)] length = N
    Returns:
        (energy_real_fp16, energy_imag_fp16)  # imag ~ 0
    """
    acc = [np.float16(0.0), np.float16(0.0)]
    for z in frame_pairs:
        z_conj = cconj(z)                 # conj(z)
        prod = cmul(z, z_conj)            # z * conj(z) = |z|^2 + j*0
        acc = cadd(acc, prod)             # accumulate
    return acc  # real part is the energy

# =============================================================================
# Main pipeline
# =============================================================================

def build_frames(num_frames=10, N=128, base_freq_bin=5, noise_sigma=0.05,
                 event_frames=1, event_type="auto", seed=42):
    """
    Build 'num_frames' frames. Base is a complex sinusoid + AWGN.
    Insert OFDM or Chirp bursts into 'event_frames' randomly chosen frames.
    """
    random.seed(seed)
    np.random.seed(seed)

    frames = []
    gt_event_idx = sorted(random.sample(range(num_frames), k=event_frames))

    for f in range(num_frames):
        # Base complex sinusoid
        re, im = complex_sinusoid(N, freq_bin=base_freq_bin, amp=0.3, phase=random.random()*2*np.pi)
        re, im = add_awgn(re, im, sigma=noise_sigma)

        # Optionally add event
        if f in gt_event_idx:
            if event_type == "auto":
                et = random.choice(["OFDM", "Chirp"])
            else:
                et = event_type

            if et == "OFDM":
                e_re, e_im = ofdm_like(N, num_active=16, amp=0.8)
            else:
                # Chirp sweeping within a safe bin range
                e_re, e_im = chirp_linear(N, f0_bin=2, f1_bin=N//3, amp=0.8)

            re = re + e_re
            im = im + e_im

        # Quantize to fp16 pair list
        frames.append(to_fp16_pair_array(re, im))

    return frames, gt_event_idx

def detect_frames(frames, thresh=None, auto_factor=2.5):
    """
    Run ED per frame and apply threshold test.
    - thresh=None: use robust automatic threshold = median(energies) * auto_factor
    Returns:
        energies_fp16: list of np.float16, per-frame energy (real part)
        decisions_fp16: list of np.float16 (1.0 if E>thresh else 0.0)
        thresh_fp16: chosen threshold (np.float16)
    """
    energies = []
    for fr in frames:
        acc_re, acc_im = ed_sum_abs2(fr)
        # acc_re is np.float16, acc_im ~ 0
        energies.append(float(acc_re))  # keep float for stats

    energies = np.array(energies, dtype=np.float32)
    if thresh is None:
        median_E = float(np.median(energies))
        thresh_val = median_E * float(auto_factor)
    else:
        thresh_val = float(thresh)

    decisions = (energies > thresh_val).astype(np.float32)

    # Cast to fp16 for storage
    energies_fp16 = [np.float16(e) for e in energies.tolist()]
    decisions_fp16 = [np.float16(d) for d in decisions.tolist()]
    thresh_fp16 = np.float16(thresh_val)
    return energies_fp16, decisions_fp16, thresh_fp16

if __name__ == "__main__":
    # ---------------------------- Configuration ----------------------------
    NUM_FRAMES = 8
    N_PER_FRAME = 128
    BASE_FREQ_BIN = 5          # base sinusoid frequency bin
    NOISE_SIGMA = 0.05         # AWGN std
    EVENT_FRAMES = 1           # number of frames containing OFDM/Chirp
    EVENT_TYPE = "auto"        # "auto" | "OFDM" | "Chirp"
    RNG_SEED = 123             # for reproducibility

    INPUT_PATH = "../PE/DATA/ss_input.txt"
    ENERGY_PATH = "../PE/DATA/ss_energy.txt"
    DETECT_PATH = "../PE/DATA/ss_detect.txt"

    # ---------------------------- Synthesis ----------------------------
    frames, gt_idx = build_frames(num_frames=NUM_FRAMES,
                                  N=N_PER_FRAME,
                                  base_freq_bin=BASE_FREQ_BIN,
                                  noise_sigma=NOISE_SIGMA,
                                  event_frames=EVENT_FRAMES,
                                  event_type=EVENT_TYPE,
                                  seed=RNG_SEED)

    # Flatten frames to store as a single input stream (frame-major order)
    flat_input = []
    for fr in frames:
        flat_input.extend(fr)


In [4]:
store_binary(flat_input, '/net/badwater/z/hyunwon/Documents/_GIT/PROWESS/SIM/PE/DATA/spectrum_input.txt')

In [5]:
# ---------------------------- ED + Detection ----------------------------
energies_fp16, decisions_fp16, thresh_fp16 = detect_frames(frames,
                                                            thresh=None,
                                                            auto_factor=2.5)

In [6]:
thresh_fp16

31.02

In [7]:
binary_to_integer(fp16_to_binary(thresh_fp16), 16)

20417

In [ ]:

    # Store input as 32-bit lines (re16||im16) using provided util
    store_binary(flat_input, INPUT_PATH)

    # ---------------------------- ED + Detection ----------------------------
    energies_fp16, decisions_fp16, thresh_fp16 = detect_frames(frames,
                                                               thresh=None,
                                                               auto_factor=2.5)

    # Store per-frame energy (one 16-bit value per line)
    store_binary(energies_fp16, ENERGY_PATH)

    # Store decisions as 16-bit (1.0 / 0.0)
    store_binary(decisions_fp16, DETECT_PATH)

    # ---------------------------- Console report ----------------------------
    print(f"[Ground truth event frames] {gt_idx}")
    print(f"[Threshold (auto)] {float(thresh_fp16):.6f}")
    for i, (e, d) in enumerate(zip(energies_fp16, decisions_fp16)):
        print(f"Frame {i:02d}: E = {float(e):.6f}  ->  DETECT = {int(d)}")
